In [9]:
import os
import sys
import cv2 as cv
import seaborn as sns
import utilities as ut
from torch.utils.data import DataLoader
import random
random.seed(0)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os,sys
import random
import torch
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torch.utils.data import DataLoader
import math
import cv2 as cv

from torch.nn import Linear, Conv2d, BatchNorm2d, MaxPool2d, Dropout2d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax
from torch.nn import Linear, GRU, Conv2d, Dropout, MaxPool2d, BatchNorm1d

# Encoding the dataset

**Creating a datframe to store paths of the X and y**

In [10]:
path_to_folder = "/Users/christianpederjacobsen/Dropbox/Mac/Desktop/leg/brain_cancer_seg/data/archive/kaggle_3m"
image_df = ut.generate_path_dataframe("/Users/christianpederjacobsen/Dropbox/Mac/Desktop/leg/brain_cancer_seg/data/archive/kaggle_3m")
image_df = image_df.sample(frac=1, random_state=1).reset_index(drop=True)
image_df

,patient_slice,mask,image,patient,diagnosis,slice
0,TCGA_FG_5962_20000626_2,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,TCGA_FG_5962_20000626,0,2
1,TCGA_DU_A5TP_19970614_1,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,TCGA_DU_A5TP_19970614,0,1
2,TCGA_DU_7294_19890104_8,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,TCGA_DU_7294_19890104,0,8
3,TCGA_DU_8165_19970205_31,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,TCGA_DU_8165_19970205,0,31
4,TCGA_DU_8166_19970322_10,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,TCGA_DU_8166_19970322,0,10
...,...,...,...,...,...,...
3924,TCGA_HT_7608_19940304_4,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,TCGA_HT_7608_19940304,0,4
3925,TCGA_HT_7881_19981015_69,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,TCGA_HT_7881_19981015,0,69
3926,TCGA_FG_6689_20020326_6,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,TCGA_FG_6689_20020326,0,6
3927,TCGA_FG_6688_20020215_34,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,/Users/christianpederjacobsen/Dropbox/Mac/Desk...,TCGA_FG_6688_20020215,0,34


**Encoding the dataset** 

In [11]:
encoded_images = np.array([cv.imread(x) for x in  image_df["image"]]).astype('float32')
encoded_masks = np.array([cv.imread(x) for x in  image_df["mask"]]).astype('float32')

encoded_images = np.moveaxis(encoded_images, -1, 1)
encoded_masks = np.moveaxis(encoded_masks, -1, 1)
encoded_masks = encoded_masks[:,0,:,:]
encoded_masks = np.expand_dims(encoded_masks, 1)

In [12]:
batch_size = 32
train_index, val_index = ut.generate_train_val_indexes(0.9,len(encoded_images))

# The training dataset
images_train_loader = list(DataLoader(encoded_images[train_index],batch_size=batch_size))
mask_train_loader = list(DataLoader(encoded_masks[train_index],batch_size=batch_size))

# The validation dataset
images_val_loader = list(DataLoader(encoded_images[val_index],batch_size=batch_size))
mask_val_loader = list(DataLoader(encoded_masks[val_index],batch_size=batch_size))

**Importing the dataset**

In [21]:
# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device("mps")
print('Device state:', device)

Device state: mps


In [20]:
%%timeit
from model_structures import Unet,get_n_params
image_random = np.random.normal(0,1, (32, 3, 256, 256)).astype('float32')
image_random = Variable(torch.from_numpy(image_random))
net = Unet(1)
# print(get_n_params(net))
print(net(image_random).shape)

torch.Size([32, 1, 256, 256])
torch.Size([32, 1, 256, 256])
torch.Size([32, 1, 256, 256])
torch.Size([32, 1, 256, 256])
torch.Size([32, 1, 256, 256])
torch.Size([32, 1, 256, 256])
torch.Size([32, 1, 256, 256])
torch.Size([32, 1, 256, 256])
13.5 s ± 713 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
from model_structures import Unet,get_n_params
image_random = np.random.normal(0,1, (1, 3, 256, 256)).astype('float32')
image_random = Variable(torch.from_numpy(image_random))
net = Unet(1)
net.to(device)


Unet(
  (down): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (d_conv1): DoubleConv(
    (model): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
    )
  )
  (d_conv2): DoubleConv(
    (model): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
    )
  )
  (d_conv3): DoubleConv(
    (model): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
    )
  )
  (d_conv4): DoubleConv(
    (model): Sequential(
      (0): Conv2d(256, 512, ker

In [ ]:
%%timeit
print(net(image_random).shape)

In [41]:
def train(model, device, epochs, train_loaders, val_loaders):
    # Optimizer
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(),lr=0.001)

    images_train_loader,mask_train_loader = train_loaders
    images_val_loader,mask_val_loader = val_loaders


    for epoch in range(epochs):
        running_loss = 0.0
        for i in range(len(images_train_loader)):
            images = images_train_loader[i]
            masks = mask_train_loader[i]
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(images)
            loss = criterion(outputs, masks)
            print(loss)
            loss.backward()
            optimizer.step()
            # print statistics
            running_loss += loss.item()
            if (i+1) % 10 == 0:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0



In [42]:
train_loaders = (images_train_loader,mask_train_loader)
val_loaders = (images_val_loader,mask_val_loader)
net = Unet(1)

train(net,"cpu",1,train_loaders,val_loaders)

tensor(0.9768, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
